In [1]:
import sys
sys.path.append("../") # go to parent dir

%load_ext autoreload
%autoreload 2

In [2]:
from evrp.utils.utilities import parse_csv_tables, calculate_distance_matrix, generate_index_mapping, create_plotting_edges, create_flat_distance_matrix, create_coordinate_dict, create_flat_optimal_edges, create_optimal_edges, merge_variable_results
from evrp.utils.graph import draw_plain_graph, draw_graph
from evrp.utils.plot import plot_interactive_graph
from evrp.milp.tsp import TSP
from evrp.milp.vrptw import VRPTW
from evrp.milp.evrptw import EVRPTW

import pandas as pd
import numpy as np
from scipy.spatial import distance

import pprint

import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

import matplotlib.pyplot as plt
import matplotlib.colors as pltc

import itertools

import networkx as nx

from pyomo.environ import *

pd.options.plotting.backend = "plotly"
pp = pprint.PrettyPrinter()

In [3]:
instance = 'r105C5_e'
fpath = 'config/test_instances/' + instance + '.csv'

In [417]:
m = EVRPTW()
m.solve(fpath)

2020-12-23 04:25:15 INFO     Building abstract model
2020-12-23 04:25:15 INFO     Defining parameters and sets
2020-12-23 04:25:15 INFO     Defining variables
2020-12-23 04:25:15 INFO     Defining constraints
2020-12-23 04:25:15 INFO     Defining objective
2020-12-23 04:25:15 INFO     Importing EVRPTW MILP instance: r105C5_e
2020-12-23 04:25:15 INFO     Reading CSV
2020-12-23 04:25:15 INFO     Creating graph
2020-12-23 04:25:15 INFO     Creating timeseries data
2020-12-23 04:25:15 INFO     Creating duplicates and extended graph
2020-12-23 04:25:15 INFO     Calculating distance matrix
2020-12-23 04:25:15 INFO     Creating parameters
2020-12-23 04:25:15 INFO     Creating instance
2020-12-23 04:25:15 INFO     Solving instance...
/Users/ramiariss/anaconda/envs/local_python36_env/bin/gurobi.sh: line 8: /Library/gurobi902/mac64/bin/python3.7: No such file or directory
2020-12-23 04:25:15 ERROR    Solver (gurobi) returned non-zero return code (127)
2020-12-23 04:25:15 ERROR    See the solver 

ERROR: Solver (gurobi) returned non-zero return code (127)
ERROR: See the solver log above for diagnostic information.


ApplicationError: Solver (gurobi) did not exit normally

In [407]:
m.data['V_']['tS'].to_dict()

{'D0': 0,
 'D1': 0,
 'S0_0': 0,
 'S1_0': 0,
 'S13_0': 0,
 'C91': 10,
 'C28': 10,
 'C75': 10,
 'C78': 10,
 'C95': 10}